In [1]:
import carla
import numpy as np
import cv2
import time

from IPython.display import clear_output, Image
import ipywidgets as widgets
import threading


In [15]:
# Parameters
simulator_host_ip = "192.168.1.71"
start_simulation = True

# Add a global variable to control the loop
stop_camera_loop = False

In [3]:
def process_image(image):
    img_array = np.array(image.raw_data)
    img_rgb = img_array.reshape((image.height, image.width, 4))
    img_rgb = img_rgb[:, :, :3]
    img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    img_jpeg = cv2.imencode('.jpeg', img_bgr)[1].tobytes()
    return img_jpeg

def display_image(image_data):
    clear_output(wait=True)
    display(Image(image_data))


In [11]:

remote_host_ip = simulator_host_ip
remote_host_port = 2000

client = carla.Client(remote_host_ip, remote_host_port)
client.set_timeout(10.0)

world = client.get_world()
blueprint_library = world.get_blueprint_library()

# Spawn a vehicle
vehicle_bp = blueprint_library.find('vehicle.tesla.model3')
spawn_point = world.get_map().get_spawn_points()[0]
vehicle = world.spawn_actor(vehicle_bp, spawn_point)

# Attach a camera sensor to the vehicle
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '800')
camera_bp.set_attribute('image_size_y', '600')
camera_bp.set_attribute('fov', '110')

camera_transform = carla.Transform(carla.Location(x=2.5, z=1.2))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)


In [12]:
def on_stop_button_click(button):
    global stop_camera_loop
    stop_camera_loop = True

stop_button = widgets.Button(description="Stop Simulation")
stop_button.on_click(on_stop_button_click)
display(stop_button)

Button(description='Stop Simulation', style=ButtonStyle())

In [13]:
# Process and display camera images using IPython.display
img_widget = widgets.Image(format='png')
display(img_widget)

def on_new_image(image):
    global img_widget
    image_data = process_image(image)
    img_widget.value = image_data

Image(value=b'')

In [16]:
def run_camera():
    global stop_camera_loop
    try:
        while not stop_camera_loop:
            time.sleep(1)
    except KeyboardInterrupt:
        pass
    finally:
        camera.stop()
        vehicle.destroy()

camera.listen(on_new_image)

camera_thread = threading.Thread(target=run_camera, daemon=True)
camera_thread.start()


In [17]:
def control_vehicle(vehicle, throttle, brake, steer):
    control = carla.VehicleControl()
    control.throttle = throttle
    control.brake = brake
    control.steer = steer
    vehicle.apply_control(control)